In [1]:
#import modules
import lxml.etree as etree
import gensim
import spacy
import glob
import copy
import pickle
from multiprocessing import Pool

### If data is not preprocessed yet

In [2]:
hbbw_xmls = glob.glob('/home/phillip/Documents/university/phd/bullinger/bullinger-korpus/data/editions/hbbw/*/*.xml')
vbs_xmls = glob.glob('/home/phillip/Documents/university/phd/bullinger/bullinger-korpus/data/editions/vbs/*/*.xml')
transcribed_xmls = glob.glob('/home/phillip/Documents/university/phd/bullinger/bullinger-korpus/data/non_edited/transcribed/*.xml')

In [3]:
all_xmls = hbbw_xmls + vbs_xmls + transcribed_xmls
len(all_xmls)

8640

In [4]:
!pip3 install https://huggingface.co/latincy/la_core_web_lg/resolve/main/la_core_web_lg-any-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 13.0 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for spacy<3.7.0,>=3.6.0 from https://files.pythonhosted.org/packages/58/93/051fc55f3e4e6d0d97beb77bbd5cd9e17589a0092305b4ae6164c6dcb58a/spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
Using cached spacy-3.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
DEPRECATION: celery 4.4.7 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyth

In [7]:
nlp = spacy.load('la_core_web_trf')

In [ ]:
texts = []
for i, xml in enumerate(all_xmls):
    if i % 300 == 0:
        print('I am at file {}'.format(i))
    text = []
    la_sents = 0
    xml_open = etree.parse(open(xml, 'r'))
    xml_name = xml.split('/')[-1].replace('.xml', '') 
    root = xml_open.getroot()
    sentences = root.findall('.//s')
    for s in sentences:
        if s.get('lang') == 'la':
            s_copy = copy.deepcopy(s)
            for fl in s_copy.findall('.//fl'):
                fl.clear()
                try:
                    s_copy.remove(fl)
                except:
                    s_copy == s_copy
            
            la_sents += 1
            try:
                doc = nlp("".join(s_copy.itertext()).replace('[', '').replace(']', ''))
                lemmas = [w.lemma_ for w in doc if w.pos_ in ['NOUN']]
                text += lemmas
            except TypeError:
                continue
    if la_sents >= 5:
        texts.append([xml_name, 'latin', text])

I am at file 0
I am at file 300
I am at file 600
I am at file 900
I am at file 1200
I am at file 1500
I am at file 1800
I am at file 2100
I am at file 2400
I am at file 2700
I am at file 3000
I am at file 3300
I am at file 3600
I am at file 3900
I am at file 4200
I am at file 4500
I am at file 4800
I am at file 5100
I am at file 5400
I am at file 5700
I am at file 6000
I am at file 6300
I am at file 6600
I am at file 6900


In [21]:
texts

[['05_606_10644',
  'latin',
  ['Doctor',
   'cornmendatio',
   'uir',
   'res',
   'laus',
   'autoritati',
   'res',
   'ludus',
   'nonae',
   'iulius',
   'Bullingerus',
   'contionator',
   'praeceptor',
   'dominus']]]

In [31]:
len(texts)

7374

In [33]:
# texts all in a list, in the form of [[{file_name: ['tok1', 'tok2', ...]}], [{file_name: ['tok1', 'tok2', ...]}], ...]
with open('bullinger-letters-tok-lem.list', 'wb') as file:
    pickle.dump(texts, file)

### If data is already pre-processed and in the format mentioned above

In [4]:
texts = pickle.load(open('data/bullinger-letters-tok-lem.list', 'rb'))

In [5]:
values = [value_list for inner_list in texts for dictionary in inner_list for value_list in dictionary.values()]

In [6]:
dictionary = gensim.corpora.Dictionary(values)
corpus = [dictionary.doc2bow(doc) for doc in values]

In [ ]:
# also save corpus as a mallet format


In [7]:
# Train LDA model
num_topics = 50  # You can adjust this based on your data and requirements
lda = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=num_topics, passes=200, workers=16)

# Print the topics
for idx, topic in lda.print_topics(-1):
    print(f"Topic: {idx} \nWords: {topic}\n")

Topic: 0 
Words: 0.069*"dominus" + 0.041*"frater" + 0.026*"Christus" + 0.021*"Bullingerus" + 0.019*"nihil" + 0.014*"annus" + 0.013*"uir" + 0.012*"Heinricho" + 0.012*"animus" + 0.011*"res"

Topic: 1 
Words: 0.007*"liber" + 0.006*"translatio" + 0.006*"Clemens" + 0.005*"textus" + 0.005*"consul" + 0.004*"uersio" + 0.004*"carmen" + 0.004*"scriptor" + 0.004*"census" + 0.003*"epistola"

Topic: 2 
Words: 0.008*"majestas" + 0.007*"parliamentum" + 0.006*"Jesus" + 0.004*"synodus" + 0.004*"Joannes" + 0.004*"tempus" + 0.003*"litera" + 0.003*"ciuis" + 0.003*"Cromwellus" + 0.003*"libatio"

Topic: 3 
Words: 0.022*"annus" + 0.021*"uir" + 0.021*"res" + 0.020*"dominus" + 0.016*"litera" + 0.013*"urbs" + 0.013*"episcopus" + 0.012*"Uadianus" + 0.012*"nomen" + 0.012*"Bullingerus"

Topic: 4 
Words: 0.037*"dominus" + 0.031*"ecclesia" + 0.029*"uir" + 0.024*"res" + 0.024*"litera" + 0.022*"frater" + 0.018*"animus" + 0.015*"Christus" + 0.013*"deus" + 0.012*"Bullingerus"

Topic: 5 
Words: 0.013*"dies" + 0.009*"libe

In [10]:
lda.save("models/gensim_lda_model-bullinger_nouns_50topics/gensim_lda_model-bullinger_nouns_50topics")